In [14]:
from dataclasses import dataclass
from typing import Callable, Dict

@dataclass 
class Tool:
    name: str
    description: str
    call: Callable[..., str]  # kwargs in, string out 

REGISTRY : Dict[str, Tool] = {}

# local mock tools (offline-friendly)
def register(tool: Tool) -> None:
    REGISTRY[tool.name] = tool

def weather_tool(city: str) -> str:
    return f"weather in {city}: 22 degrees, clear"

def wiki_tool(topic: str) -> str:
    return f"(mock) Quick facts about {topic}: ..."

#register the weather tool
register(
    Tool(
        name = "weather", 
        description="Get current weather by city", 
        call = lambda city: weather_tool(city)
    )
)
#register the wiki tool
register(
    Tool(
        name = "wikipedia", 
        description="Short summary of a topic", 
        call = lambda topic: wiki_tool(topic)
    )
)

In [6]:
print(REGISTRY)

{'weather': Tool(name='weather', description='Get current weather by city', call=<function <lambda> at 0x152752fc0>), 'wikipedia': Tool(name='wikipedia', description='Short summary of a topic', call=<function <lambda> at 0x1527528e0>)}


# router 
def choose_tool()


def find_city()

def find_topic() 

# orchestration 
def run() :
    if tool == "weather":
        city = find_city()
        return

    if tool == "wikipedia"
        tool = find_topic()
        return
    
    return 

In [10]:
from typing import Tuple, Optional
#from agent.tools import REGISTRY

# --- 1) Router: pick a tool name from a query (tiny, explicit) ---
def choose_tool(query: str) -> Tuple[str, str]:
    q = query.lower()
    if "weather" in q or "forecast" in q:
        return "weather", "matched keyword: weather/forecast"
    if any(kw in q for kw in ["wiki", "wikipedia", "who is", "what is"]):
        return "wikipedia", "matched keyword: wiki/who/what"
    return "llm", "no keyword match"

# --- 2) Naive argument extraction (kept simple for Lab 1) ---
def _find_city(query: str) -> Optional[str]:
    q = query.lower()
    cities = [
        "san francisco", "new york",
        "chicago", "paris", "london", "mumbai", "tokyo", "delhi", "bangalore"
    ]
    for c in cities:
        if c in q:
            return c.title()
    return None

def _find_topic(query: str) -> str:
    q = query.strip()
    lower = q.lower()
    if "about" in lower:
        return q[lower.index("about") + len("about"):].strip() or "Agentic AI"
    if "on " in lower:
        return q[lower.index("on ") + len("on "):].strip() or "Agentic AI"
    return q

# --- 3) Orchestrator: route -> extract args -> call tool (or fallback) ---
def run_once(query: str) -> str:
    tool, reason = choose_tool(query)

    if tool == "weather":
        city = _find_city(query)
        if not city:
            return "I can check the weather, but I couldn't spot the city. Try: 'weather in Paris'."
        out = REGISTRY["weather"].call(city=city)  
        return f"[weather] {out}  (reason: {reason})"

    if tool == "wikipedia":
        topic = _find_topic(query)
        out = REGISTRY["wikipedia"].call(topic=topic)  
        return f"[wikipedia] {out}  (reason: {reason})"

    return "(llm) I'd answer conversationally here. (reason: no matching tool)"
# receive query --> calls router --> extract --> calls tools --> return answer 

In [16]:
run_once("what is the weather in paris today?")

'[weather] weather in Paris: 22 degrees, clear  (reason: matched keyword: weather/forecast)'

In [18]:
run_once("tell me about LLMs")

"(llm) I'd answer conversationally here. (reason: no matching tool)"

In [22]:
run_once("tell me about LLMs from wiki")

'[wikipedia] (mock) Quick facts about LLMs from wiki: ...  (reason: matched keyword: wiki/who/what)'